In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark.sql.functions import col, hour, minute, second
from pyspark.sql.types import TimestampType, DoubleType, IntegerType

In [ ]:
import subprocess

args = "hdfs dfs -ls | awk '{print $8}'"
proc = subprocess.Popen(args, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)

s_output, s_err = proc.communicate()
all_dart_dirs = s_output.split()
all_dart_dirs = all_dart_dirs[1:]

In [ ]:
! mkdir output

In [ ]:
from pyspark.sql import SparkSession

spark = (
    SparkSession
        .builder
        .appName("OTUS_hw3")
        .config("spark.dynamicAllocation.enabled", "true")
        .config("spark.executor.memory", "2g")
        .config("spark.driver.memory", "1g")
        .getOrCreate()
)

Типы некорректных данных:
1. (обнаружено) сustomer_id=-999999, сustomer_id=999999
2. (обнаружено) tx_amount=0
3. (обнаружено) transaction_id: дубликаты
4. (проверка) tx_datetime: значения, которые выпадают за требуемые ограничения (часы, минуты и т.п.)
5. (проверка) tx_fraud: вне [0, 1], tx_fraud_scenario: вне [0, 3]

### Создать скрипт, который должен выполнять очистку данных с использованием Apache Spark:

In [ ]:
def clean_data(path):
    data = spark.read.text(path)
    header=data.first()[0]
    schema=header.split(" | ")
    df_new = data.filter(data["value"] != header).rdd.map(lambda x:x[0].split(",")).toDF(schema)
    df_new = df_new.withColumn("tx_datetime", df_new.tx_datetime.cast(TimestampType()))\
        .withColumn("customer_id", df_new.customer_id.cast(IntegerType()))\
        .withColumn("terminal_id", df_new.terminal_id.cast(IntegerType()))\
        .withColumn("tx_amount", df_new.tx_amount.cast(DoubleType()))\
        .withColumn("tx_time_seconds", df_new.tx_time_seconds.cast(IntegerType()))\
        .withColumn("tx_time_days", df_new.tx_time_days.cast(IntegerType()))\
        .withColumn("tx_fraud", df_new.tx_fraud.cast(IntegerType()))\
        .withColumn("tx_fraud_scenario", df_new.tx_fraud_scenario.cast(IntegerType()))
    
    df_new = df_new.withColumn("hour", hour(col("tx_datetime")))\
        .withColumn("minute", minute(col("tx_datetime")))\
        .withColumn("second", second(col("tx_datetime")))

    df_new = df_new.dropDuplicates(["# tranaction_id"])
    df_new = df_new.filter((df_new["customer_id"] != -999999) & 
                           (df_new["customer_id"] != 999999) &
                           (df_new["tx_amount"] != 0)
                          )
    
    data.write.parquet("output/" + path[:-4] + ".parquet")

### Выполнить очистку датасета с использованием созданного скрипта и сохранить его в bucket’е в формате parquet:

In [ ]:
for path in all_dart_dirs:
    clean_data(str(path)[2:-1])

In [ ]:
spark.stop()